In [ ]:
# %% [markdown]
# # Task 1: Exploratory Data Analysis and Data Preprocessing
# 
# ## Objective
# Understand the structure, content, and quality of the complaint data and prepare it for the RAG pipeline.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from collections import Counter
import json
from datetime import datetime

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# %%
# Load the full CFPB complaint dataset
print("Loading dataset...")
df = pd.read_csv('../data/raw/complaints.csv', low_memory=False)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# %%
# Initial EDA - Display basic info
print("Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
display(df.head())

# %%
# Check for missing values
print("Missing values per column:")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})
display(missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage', ascending=False))

# %%
# Analyze the distribution of complaints across different Products
print("Product distribution:")
product_counts = df['Product'].value_counts()
product_percentages = (product_counts / len(df)) * 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
ax1.bar(range(len(product_counts[:10])), product_counts.values[:10])
ax1.set_xlabel('Product')
ax1.set_ylabel('Number of Complaints')
ax1.set_title('Top 10 Products by Complaint Count')
ax1.set_xticks(range(len(product_counts[:10])))
ax1.set_xticklabels(product_counts.index[:10], rotation=45, ha='right')

# Pie chart for top 10
ax2.pie(product_counts.values[:10], labels=product_counts.index[:10], autopct='%1.1f%%')
ax2.set_title('Product Distribution (Top 10)')

plt.tight_layout()
plt.show()

# %%
# Filter for our five specified products
target_products = [
    'Credit card',
    'Credit card or prepaid card',
    'Credit reporting',
    'Debt collection',
    'Money transfer, virtual currency, or money service',
    'Virtual currency'
]

# Let's see what products we actually have
print("All unique products in dataset:")
for product in df['Product'].unique():
    print(f"  - {product}")

# %%
# Based on the dataset, let's identify the correct product names
# Looking at typical CFPB dataset, we need to map to our requirements
product_mapping = {
    'Credit card': ['Credit card', 'Credit card or prepaid card'],
    'Personal loan': ['Payday loan', 'Student loan', 'Vehicle loan or lease'],
    'Savings account': ['Bank account or service', 'Checking or savings account'],
    'Money transfer': ['Money transfer, virtual currency, or money service', 'Virtual currency']
}

# Let's check what we have
all_products = []
for category, products in product_mapping.items():
    for product in products:
        if product in df['Product'].unique():
            all_products.extend(products)
            break

# %%
# Calculate and visualize the length of Consumer complaint narrative
print("Analyzing complaint narrative length...")

# Check if we have the narrative column
if 'Consumer complaint narrative' in df.columns:
    # Calculate word count for each narrative
    df['narrative_word_count'] = df['Consumer complaint narrative'].fillna('').apply(lambda x: len(str(x).split()))
    
    # Create histogram of narrative lengths
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    ax1.hist(df['narrative_word_count'], bins=50, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Word Count')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Complaint Narrative Length')
    ax1.axvline(df['narrative_word_count'].median(), color='red', linestyle='--', label=f'Median: {df["narrative_word_count"].median():.0f}')
    ax1.axvline(df['narrative_word_count'].mean(), color='green', linestyle='--', label=f'Mean: {df["narrative_word_count"].mean():.0f}')
    ax1.legend()
    ax1.set_xlim(0, 1000)  # Limit x-axis for better visualization
    
    # Box plot
    ax2.boxplot(df['narrative_word_count'].dropna())
    ax2.set_ylabel('Word Count')
    ax2.set_title('Box Plot of Narrative Length')
    ax2.set_xticklabels(['Narratives'])
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("Narrative Length Statistics:")
    print(f"  Mean: {df['narrative_word_count'].mean():.2f}")
    print(f"  Median: {df['narrative_word_count'].median():.2f}")
    print(f"  Std: {df['narrative_word_count'].std():.2f}")
    print(f"  Min: {df['narrative_word_count'].min():.2f}")
    print(f"  Max: {df['narrative_word_count'].max():.2f}")
    
    # Count narratives with different lengths
    print("\nNarrative Length Categories:")
    print(f"  Empty narratives: {len(df[df['Consumer complaint narrative'].isna()])}")
    print(f"  Very short (1-10 words): {len(df[(df['narrative_word_count'] > 0) & (df['narrative_word_count'] <= 10)])}")
    print(f"  Short (11-50 words): {len(df[(df['narrative_word_count'] > 10) & (df['narrative_word_count'] <= 50)])}")
    print(f"  Medium (51-200 words): {len(df[(df['narrative_word_count'] > 50) & (df['narrative_word_count'] <= 200)])}")
    print(f"  Long (201-500 words): {len(df[(df['narrative_word_count'] > 200) & (df['narrative_word_count'] <= 500)])}")
    print(f"  Very long (>500 words): {len(df[df['narrative_word_count'] > 500])}")

# %%
# Identify the number of complaints with and without narratives
if 'Consumer complaint narrative' in df.columns:
    has_narrative = df['Consumer complaint narrative'].notna() & (df['Consumer complaint narrative'].str.strip() != '')
    print("Complaints with narratives:")
    print(f"  With narrative: {has_narrative.sum()} ({has_narrative.mean()*100:.2f}%)")
    print(f"  Without narrative: {(~has_narrative).sum()} ({((~has_narrative).mean()*100):.2f}%)")
    
    # Visualize
    plt.figure(figsize=(8, 6))
    has_narrative.value_counts().plot(kind='pie', autopct='%1.1f%%', 
                                      labels=['Without Narrative', 'With Narrative'],
                                      colors=['#ff9999', '#66b3ff'])
    plt.title('Complaints With vs Without Narratives')
    plt.ylabel('')
    plt.show()

# %%
# Filter the dataset to meet project requirements
print("Filtering dataset...")

# First, let's see what columns we have
print("Available columns:", df.columns.tolist())

# Based on typical CFPB dataset structure
# We'll create a function to map to our required categories
def map_to_target_category(product):
    product = str(product).lower()
    
    if any(keyword in product for keyword in ['credit card', 'prepaid card']):
        return 'Credit Cards'
    elif any(keyword in product for keyword in ['loan', 'mortgage']):
        return 'Personal Loans'
    elif any(keyword in product for keyword in ['savings', 'checking', 'bank account', 'deposit']):
        return 'Savings Accounts'
    elif any(keyword in product for keyword in ['money transfer', 'money service', 'virtual currency', 'wire transfer']):
        return 'Money Transfers'
    else:
        return 'Other'

# Apply mapping
df['product_category'] = df['Product'].apply(map_to_target_category)

# Check distribution
print("Mapped product categories distribution:")
category_counts = df['product_category'].value_counts()
display(category_counts)

# %%
# Filter for our four target categories
target_categories = ['Credit Cards', 'Personal Loans', 'Savings Accounts', 'Money Transfers']
filtered_df = df[df['product_category'].isin(target_categories)].copy()
print(f"Filtered dataset shape: {filtered_df.shape}")
print(f"Original dataset shape: {df.shape}")
print(f"Percentage kept: {len(filtered_df)/len(df)*100:.2f}%")

# %%
# Remove records with empty narratives
if 'Consumer complaint narrative' in filtered_df.columns:
    initial_count = len(filtered_df)
    filtered_df = filtered_df[
        filtered_df['Consumer complaint narrative'].notna() & 
        (filtered_df['Consumer complaint narrative'].str.strip() != '')
    ].copy()
    print(f"Removed {initial_count - len(filtered_df)} records with empty narratives")
    print(f"Final filtered dataset shape: {filtered_df.shape}")

# %%
# Text cleaning function
def clean_text(text):
    """
    Clean the text narratives to improve embedding quality
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Lowercase
    text = text.lower()
    
    # Remove excessive whitespace
    text = ' '.join(text.split())
    
    # Remove common boilerplate text patterns
    boilerplate_patterns = [
        r'i am writing to file a complaint',
        r'to whom it may concern',
        r'dear sir/madam',
        r'this is a complaint regarding',
        r'please be advised that',
        r'i am writing to express my dissatisfaction',
        r'i would like to file a complaint',
        r'complaint id:',
        r'reference number:',
        r'case number:',
    ]
    
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove special characters but keep basic punctuation
    text = re.sub(r'[^\w\s.,!?]', ' ', text)
    
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    return text

# %%
# Apply text cleaning
print("Cleaning text narratives...")
filtered_df['cleaned_narrative'] = filtered_df['Consumer complaint narrative'].apply(clean_text)

# Check cleaning results
print("\nSample before cleaning:")
sample_idx = filtered_df.index[0]
print(filtered_df.loc[sample_idx, 'Consumer complaint narrative'][:500])
print("\nSample after cleaning:")
print(filtered_df.loc[sample_idx, 'cleaned_narrative'][:500])

# %%
# Check word count distribution after cleaning
filtered_df['cleaned_word_count'] = filtered_df['cleaned_narrative'].apply(lambda x: len(x.split()))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(filtered_df['cleaned_word_count'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Word Count (Cleaned)')
plt.ylabel('Frequency')
plt.title('Distribution of Cleaned Narrative Length')
plt.axvline(filtered_df['cleaned_word_count'].median(), color='red', linestyle='--', 
           label=f'Median: {filtered_df["cleaned_word_count"].median():.0f}')
plt.legend()

plt.subplot(1, 2, 2)
category_means = filtered_df.groupby('product_category')['cleaned_word_count'].mean()
category_means.plot(kind='bar')
plt.xlabel('Product Category')
plt.ylabel('Average Word Count')
plt.title('Average Narrative Length by Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# %%
# Save the cleaned and filtered dataset
output_path = 'data/processed/filtered_complaints.csv'
filtered_df.to_csv(output_path, index=False)
print(f"Saved filtered dataset to: {output_path}")
print(f"Final dataset size: {len(filtered_df)} complaints")

# %%
# Summary statistics
print("\n" + "="*50)
print("FINAL DATASET SUMMARY")
print("="*50)
print(f"Total complaints: {len(filtered_df)}")
print("\nComplaints by product category:")
for category, count in filtered_df['product_category'].value_counts().items():
    percentage = (count / len(filtered_df)) * 100
    print(f"  {category}: {count} ({percentage:.1f}%)")

print(f"\nAverage word count per narrative: {filtered_df['cleaned_word_count'].mean():.1f}")
print(f"Median word count: {filtered_df['cleaned_word_count'].median():.1f}")
print(f"Date range: {filtered_df['Date received'].min()} to {filtered_df['Date received'].max()}")

# Check for duplicates
duplicate_narratives = filtered_df['cleaned_narrative'].duplicated().sum()
print(f"\nDuplicate narratives: {duplicate_narratives} ({duplicate_narratives/len(filtered_df)*100:.2f}%)")

# %%
# Additional analysis: Most common issues by category
print("\nMost common issues by product category:")
for category in target_categories:
    category_data = filtered_df[filtered_df['product_category'] == category]
    if 'Issue' in category_data.columns:
        top_issues = category_data['Issue'].value_counts().head(5)
        print(f"\n{category}:")
        for issue, count in top_issues.items():
            print(f"  - {issue}: {count}")

# %%
# Save a sample for quick testing
sample_df = filtered_df.sample(min(1000, len(filtered_df)), random_state=42)
sample_path = 'data/processed/sample_complaints.csv'
sample_df.to_csv(sample_path, index=False)
print(f"\nSaved sample dataset to: {sample_path}")

Loading dataset...
